In [5]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field
from google.oauth2.credentials import Credentials
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
tavily_client = TavilyClient(api_key=tavily_key)
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))
GEMINI_MODEL='gemini-2.0-flash'
langchain_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)


In [1]:
from token_creator import get_creds

In [25]:
creds=get_creds()

In [27]:

# if os.path.exists("token.json"):
#     creds = Credentials.from_authorized_user_file("token.json")
creds=Credentials.from_authorized_user_info(info={'refresh_token':"",
                                                  'client_id':"",
                                                  'client_secret':""},
                                                  scopes=["https://mail.google.com/", "https://www.googleapis.com/auth/calendar", "https://www.googleapis.com/auth/cloud-platform", "https://www.googleapis.com/auth/contacts", "https://www.googleapis.com/auth/tasks", "https://www.googleapis.com/auth/generative-language.retriever"])
api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'creds':creds
}


In [28]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Deps:
    deep_research_output: dict

In [11]:
class Cortana_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use as a dictionary
        """
        pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
        GEMINI_MODEL='gemini-2.0-flash'
        langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
        # tools
        google_agent=Google_agent(langchain_llm,api_keys)
        deep_research_engine=Deep_research_engine(pydantic_llm,api_keys)
        
        def google_agent_tool(query:str):
            """
            Use this tool to interact with the google agent, which can, search for images, manage the user's calendar, emails, google tasks, contacts, and more.
            Args:
                query (str): The entire query related to the google agent and its capabilities
            Returns:
                str: The response from the google agent
            """
            return google_agent.chat(query)

       

        async def search_and_question_answering_tool(ctx: RunContext[Deps], query:str):
            """
            Use this tool to do a deep research on a topic, to gather detailed informations and data, answer_questions from the deep research results or do a quick research if the answer is not related to the deep research.
            Args:
                query (str): The query related to the search_and_question_answering_tool and its capabilities
                

            Returns:
                str: The response from the search_and_question_answering_tool
            """
            @dataclass
            class Route:
                answer: str = Field(default_factory=None,description="the answer to the question if the question is related to the deep research")
                route: str = Field(description="the route, either deep_research or answer_question, or quick_research")
            agent=Agent(pydantic_llm, result_type=Route, system_prompt="you are a router/question answering agent, you are given a query and you need to decide what to do based on the information provided")
            response=await agent.run(f"based on the query: {query}, and the information provided: {ctx.deps.deep_research_output if ctx.deps.deep_research_output else ''} either answer the question or if the answer is not related to the information provided or need more information, return 'quick_research' or 'deep_research'")
            route=response.data.route
            if route=='deep_research':
                response=await deep_research_engine.chat(query)
                ctx.deps.deep_research_output=response
                return response
            elif route=='answer_question':
                
                return response.data.answer
            elif route=='quick_research':
                quick_research_agent=Agent(pydantic_llm, tools=[tavily_search_tool(api_keys['tavily_key'])], system_prompt="do a websearch based on the query")
                result=await quick_research_agent.run(query)
                return result.data

        def get_current_time_tool():
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
        
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        

        self.agent=Agent(pydantic_llm, tools=[google_agent_tool, search_and_question_answering_tool, get_current_time_tool], system_prompt="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you to help the user with their queries")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={})
    async def chat(self, query:str):
        result=await self.agent.run(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data


In [29]:
cortana_agent=Cortana_agent(api_keys)

In [30]:
print(await cortana_agent.chat('get my mail'))

OK. Here are the 5 latest emails in your inbox:
* From: Vessi <hello@vessi.com> - Subject: 💧 Limited-edition spring styles are here
* From: A10 Kitchen <promotions@spotapps.co> - Subject: A10 Kitchen
* From: LinkedIn <messages-noreply@linkedin.com> - Subject: Tammy Wolfson sent you a message 3 days ago
* From: Apollo <hello@mail.apollo.io> - Subject: ⚡ Prospect anywhere with the Apollo Chrome Extension
* From: Glassdoor Jobs <noreply@glassdoor.com> - Subject: Director, Machine Learning Engineering at Warner Bros. Discovery and 7 more jobs in New York, NY for you. Apply Now.



In [24]:
cortana_agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are Cortana, a helpful assistant that can help with a wide range of tasks,                          you can use the tools provided to you to help the user with their queries', timestamp=datetime.datetime(2025, 4, 16, 16, 0, 50, 748294, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='what can you tell me about the moon landing', timestamp=datetime.datetime(2025, 4, 16, 16, 0, 50, 748302, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='search_and_question_answering_tool', args={'query': 'what happened during the moon landing'}, tool_call_id='pyd_ai_3b5c865126574fb6ba82ffaada33ef31', part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 4, 16, 16, 0, 51, 428864, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='search_and_question_ans